# Lab 5: Large-scale music data

**Roman Tsukanov**  
SMC, 2016

*http://www.dtic.upf.edu/~aporter/amplab/*

## Creating dataset(s) based on Last.fm tags

> **acousticbrainz-2015-01-lastfm-tags.csv.bz2 (69MB)**  
> Contains tags scraped from Last.fm according to the track.getTopTags method.  
Each row contains the MBID of a recording in the first column followed by pairs of data. The first item in a pair is a tag, and the second item is its count in that recording (see their documentation). These counts are normalised per recording, that is, a count of 100 means that it is the most common tag in that recording, not that it has been tagged with this tag 100 times.  
Many of these tags could be used for genres or moods. Be careful, there are a lot of useless tags in this dataset.

In [1]:
LASTFM_TAGS_FILE = "acousticbrainz-2015-01-lastfm-tags.csv"

In [2]:
# The usual...
import matplotlib.pyplot as plt
import numpy as np
import pandas
import pprint
%matplotlib inline
%config InlineBackend.figure_formats = ['retina']

In [3]:
from itertools import izip
from collections import defaultdict
import os.path

recordings = {}  # each recording is mapped to a list of (tag, normalized count) tuples
tags = defaultdict(list)  # each tag is mapped to a list of (mbid, normalized count) tuples

def pairwise(iterable):
    i = iter(iterable)
    return izip(i, i)

with open(LASTFM_TAGS_FILE, 'r') as tags_file:
    for line in tags_file:
        line_list = line.strip().split(',')
        mbid = line_list[0]
        mbid_tags = []
        for tag, count_norm in pairwise(line_list[1:]):
            tag = tag.strip().lower()
            mbid_tags.append((tag, count_norm))
            tags[tag].append((mbid, count_norm))
        recordings[mbid] = mbid_tags

### Looking at the data

In [4]:
print("Examples:")
print("\n02aee867-c503-4749-a877-5c923a6e0908:")
pprint.pprint(recordings["02aee867-c503-4749-a877-5c923a6e0908"])
print("\n00018ca8-d84c-4c3d-bcab-5f9a4b9e4e92:")
pprint.pprint(recordings["00018ca8-d84c-4c3d-bcab-5f9a4b9e4e92"])

Examples:

02aee867-c503-4749-a877-5c923a6e0908:
[('japanese', '100'),
 ('jpop', '100'),
 ('j-pop', '50'),
 ('female vocalists', '50'),
 ('asian', '50'),
 ('asian music', '50'),
 ('asian pop', '50')]

00018ca8-d84c-4c3d-bcab-5f9a4b9e4e92:
[('video game music', '100'),
 ('instrumental', '28'),
 ('soundtrack', '28'),
 ('classical', '14'),
 ('game music', '14'),
 ('orchestral', '14'),
 ('final fantasy', '14'),
 ('final fantasy xii', '14'),
 ('slobeat', '14')]


In [5]:
print("There are %s unique tags." % len(tags))

There are 513912 unique tags.


In [6]:
import operator
print("20 most used tags:")
tag_occurences = defaultdict(int)
for tag, recordings in tags.iteritems():
    tag_occurences[tag] = len(recordings)
tag_occurences_sorted = sorted(tag_occurences.items(), key=operator.itemgetter(1), reverse=True)
pprint.pprint(tag_occurences_sorted[0:20])

20 most used tags:
[('rock', 145779),
 ('alternative', 84933),
 ('pop', 81715),
 ('favorites', 63655),
 ('electronic', 54374),
 ('alternative rock', 53157),
 ('metal', 53069),
 ('indie', 52916),
 ('classic rock', 49091),
 ('female vocalists', 47768),
 ('beautiful', 46821),
 ('love', 45473),
 ('awesome', 44766),
 ('american', 42439),
 ('hard rock', 41520),
 ('90s', 40903),
 ('instrumental', 40201),
 ('male vocalists', 40196),
 ('00s', 39217),
 ('soundtrack', 38460)]


What else can be extracted, apart from genre?

In [7]:
print("Quality of content - Good: %s, Bad: %s" % (
        tag_occurences["good"] + tag_occurences["awesome"] + tag_occurences["amazing"],
        tag_occurences["bad"] + tag_occurences["awful"] + tag_occurences["terrible"],
))
print("Mood - Sad: %s, Happy: %s" % (
        tag_occurences["sad"],
        tag_occurences["happy"]
))
print("Female/Male - Female: %s, Male: %s" % (
        tag_occurences["female vocalists"] + tag_occurences["female"],
        tag_occurences["male vocalists"] + tag_occurences["male"],
))
print("Origin - American: %s, British: %s, German: %s, Spanish: %s" % (
        tag_occurences["american"],
        tag_occurences["british"],
        tag_occurences["german"],
        tag_occurences["spanish"],
))
print("Period - 20s: %s, 30s: %s, 40s: %s, 50s: %s, 60s: %s,\n\t 70s: %s, 80s: %s, 90s: %s, 2000s: %s, 2010s: %s" % (
        tag_occurences["20s"],
        tag_occurences["30s"],
        tag_occurences["40s"],
        tag_occurences["50s"],
        tag_occurences["60s"],
        tag_occurences["70s"],
        tag_occurences["80s"],
        tag_occurences["90s"],
        tag_occurences["2000s"] + tag_occurences["00s"],
        tag_occurences["2010s"] + tag_occurences["10s"],
))
print("Rating (out of 10) - 0: %s, 1: %s, 2: %s, 3: %s, 4: %s,\n\t\t\t 5: %s, 6: %s, 7: %s, 8: %s, 9: %s, 10: %s" % (
        tag_occurences["0 of 10 stars"],
        tag_occurences["1 of 10 stars"],
        tag_occurences["2 of 10 stars"],
        tag_occurences["3 of 10 stars"],
        tag_occurences["4 of 10 stars"],
        tag_occurences["5 of 10 stars"],
        tag_occurences["6 of 10 stars"],
        tag_occurences["7 of 10 stars"],
        tag_occurences["8 of 10 stars"],
        tag_occurences["9 of 10 stars"],
        tag_occurences["10 of 10 stars"],
))
print("Rating (out of 5) - 0: %s, 1: %s, 2: %s, 3: %s, 4: %s, 5: %s" % (
        tag_occurences["0 of 5 stars"],
        tag_occurences["1 of 5 stars"],
        tag_occurences["2 of 5 stars"],
        tag_occurences["3 of 5 stars"],
        tag_occurences["4 of 5 stars"],
        tag_occurences["5 of 5 stars"],
))

Quality of content - Good: 74421, Bad: 519
Mood - Sad: 21000, Happy: 16095
Female/Male - Female: 61227, Male: 43076
Origin - American: 42439, British: 37694, German: 16989, Spanish: 2314
Period - 20s: 271, 30s: 352, 40s: 556, 50s: 2901, 60s: 17808,
	 70s: 26209, 80s: 36596, 90s: 40903, 2000s: 51428, 2010s: 9579
Rating (out of 10) - 0: 2, 1: 521, 2: 3201, 3: 2075, 4: 9075,
			 5: 7539, 6: 17580, 7: 13318, 8: 19426, 9: 9359, 10: 15681
Rating (out of 5) - 0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0


### Creating datasets

In [8]:
DATASETS_DIR = "datasets"
if not os.path.exists(DATASETS_DIR):
    os.makedirs(DATASETS_DIR)

def get_top_recordings(tags_list, limit=None):
    c_recordings = []
    for tag in tags_list:
        c_recordings += tags[tag]
    c_recordings = sorted(c_recordings, key=lambda r: r[1], reverse=True)  # sorting by normalized count
    rec_no_duplicates = []
    mbid_seen = {}
    for mbid, count in c_recordings:
        if mbid not in mbid_seen:
            mbid_seen[mbid] = True
            rec_no_duplicates.append(mbid)
    if limit:
        return rec_no_duplicates[:limit]
    return rec_no_duplicates

In [9]:
with open(os.path.join(DATASETS_DIR, "mood.csv"), "w+") as f:
    limit = 1000
    for mbid in get_top_recordings(["happy"], limit):
        f.write("%s,%s\n" % (mbid, "happy"))
    for mbid in get_top_recordings(["sad"], limit):
        f.write("%s,%s\n" % (mbid, "sad"))

In [10]:
with open(os.path.join(DATASETS_DIR, "quality.csv"), "w+") as f:
    limit = 519
    for mbid in get_top_recordings(["good", "awesome", "amazing"], limit):
        f.write("%s,%s\n" % (mbid, "good"))
    for mbid in get_top_recordings(["bad", "awful", "terrible"], limit):
        f.write("%s,%s\n" % (mbid, "bad"))

In [11]:
with open(os.path.join(DATASETS_DIR, "origin.csv"), "w+") as f:
    limit = 1000
    for mbid in get_top_recordings(["american"], limit):
        f.write("%s,%s\n" % (mbid, "american"))
    for mbid in get_top_recordings(["british"], limit):
        f.write("%s,%s\n" % (mbid, "british"))
    for mbid in get_top_recordings(["german"], limit):
        f.write("%s,%s\n" % (mbid, "german"))
    for mbid in get_top_recordings(["spanish"], limit):
        f.write("%s,%s\n" % (mbid, "spanish"))

In [12]:
with open(os.path.join(DATASETS_DIR, "period.csv"), "w+") as f:
    limit = 271
    for mbid in get_top_recordings(["20s"], limit):
        f.write("%s,%s\n" % (mbid, "20s"))
    for mbid in get_top_recordings(["30s"], limit):
        f.write("%s,%s\n" % (mbid, "30s"))
    for mbid in get_top_recordings(["40s"], limit):
        f.write("%s,%s\n" % (mbid, "40s"))
    for mbid in get_top_recordings(["50s"], limit):
        f.write("%s,%s\n" % (mbid, "50s"))
    for mbid in get_top_recordings(["60s"], limit):
        f.write("%s,%s\n" % (mbid, "60s"))
    for mbid in get_top_recordings(["70s"], limit):
        f.write("%s,%s\n" % (mbid, "70s"))
    for mbid in get_top_recordings(["80s"], limit):
        f.write("%s,%s\n" % (mbid, "80s"))
    for mbid in get_top_recordings(["90s"], limit):
        f.write("%s,%s\n" % (mbid, "90s"))
    for mbid in get_top_recordings(["2000s", "00s"], limit):
        f.write("%s,%s\n" % (mbid, "2000s"))
    for mbid in get_top_recordings(["2010s", "10s"], limit):
        f.write("%s,%s\n" % (mbid, "2010s"))

The next step is to import these datasets into AcousticBrainz and do the evaluation.

Datasets are at http://beta.acousticbrainz.org/user/Gentlecat.

## Some other random thing that I tried

Wanted to do create a dataset with genres based on MusicBrainz tags, but someone else took related task before be so... Below is everything that I've planned and managed to get done.

**Plan:**
1. Get list of unique recording MBIDs from AcousticBrainz.
2. Map them to tags on MusicBrainz (maybe somewhere else).  
    2.1 Create another table (+ schema)  
    2.2 Copy MBIDs into that table  
    2.3 Join it with tags  
    2.4 Dump a CSV file with MBIDs and associated tags
3. Create a dataset from these tags.
4. Evaluate that dataset. 
5. ???
6. 💥

### Getting unique recordings from AcousticBrainz

Using *psql* copy list of unique MBIDs submitted to AcousticBrainz:
```SQL
\copy (SELECT DISTINCT ON (mbid) mbid FROM lowlevel) TO 'mbids.txt';
```

### Mapping recordings to tags

Then with MusicBrainz database:
```SQL
CREATE SCHEMA acousticbrainz;
CREATE TABLE acousticbrainz.recordings (
	mbid UUID
);
COPY acousticbrainz.recordings FROM 'mbids.txt';
```

To get information about tags associated with recordings:
```SQL
SELECT abr.mbid, tag.name, rt.count
FROM acousticbrainz.recordings abr
JOIN musicbrainz.recording mbr ON mbr.gid = abr.mbid
JOIN musicbrainz.recording_tag rt ON rt.recording = mbr.id
JOIN musicbrainz.tag tag ON tag.id = rt.tag
LIMIT 20; -- limit is optional
```

To get 50 most popular tags associated with recordings:
```SQL
SELECT tag.name, COUNT(*) as counter
FROM acousticbrainz.recordings abr
JOIN musicbrainz.recording mbr ON mbr.gid = abr.mbid
JOIN musicbrainz.recording_tag rt ON rt.recording = mbr.id
JOIN musicbrainz.tag tag ON tag.id = rt.tag
GROUP BY tag.name
ORDER BY counter DESC
LIMIT 50;
```

_**Most of the tags on MusicBrainz are related to genre.**_